Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# 1. Подготовка данных

In [1]:
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
import timeit
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [2]:
data = pd.read_csv('/datasets/autos.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


In [3]:
data['time_in_use'] = pd.DatetimeIndex(data['DateCreated']).year - data['RegistrationYear']

графа показывающею сколько лет машина была на учете до выставления на продажу

In [4]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode,time_in_use
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087,11.765476
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248,90.227958
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000,-7983.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000,8.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000,13.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000,17.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000,1016.000000


In [5]:
data = data.drop(data[data['time_in_use']<0].index)

удалил те графы, где машина была поставленна на учет после выставления объявления

In [6]:
data = data.drop(data[data['RegistrationYear'] <1920].index)

In [7]:
data = data.drop(data[data['RegistrationYear'] > 2020].index)

будем считать машина с датой регистрации до 1920 года выбросами

In [8]:
data['Price'].describe()

count    339670.000000
mean       4472.568290
std        4546.110422
min           0.000000
25%        1100.000000
50%        2799.000000
75%        6500.000000
max       20000.000000
Name: Price, dtype: float64

In [9]:
data = data.drop(data[data['Price']<200].index)

удалил те графы, где машина c слишком низкой ценой

In [10]:
data['NotRepaired'] = data['NotRepaired'].fillna('no_info' )
data['FuelType'] = data['FuelType'].fillna('no_info' )
data['VehicleType'] = data['FuelType'].fillna('no_info' )
data['Gearbox'] = data['Gearbox'].fillna('no_info' )
data['Model'] = data['Model'].fillna('no_info' )
features_no_ohe = data[['RegistrationYear','Power','Kilometer','RegistrationMonth','NumberOfPictures','time_in_use','VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']]
target_no_ohe = data['Price']
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323224 entries, 0 to 354368
Data columns (total 17 columns):
DateCrawled          323224 non-null object
Price                323224 non-null int64
VehicleType          323224 non-null object
RegistrationYear     323224 non-null int64
Gearbox              323224 non-null object
Power                323224 non-null int64
Model                323224 non-null object
Kilometer            323224 non-null int64
RegistrationMonth    323224 non-null int64
FuelType             323224 non-null object
Brand                323224 non-null object
NotRepaired          323224 non-null object
DateCreated          323224 non-null object
NumberOfPictures     323224 non-null int64
PostalCode           323224 non-null int64
LastSeen             323224 non-null object
time_in_use          323224 non-null int64
dtypes: int64(8), object(9)
memory usage: 44.4+ MB


In [11]:
features_ohe = pd.get_dummies(data[['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']], drop_first = True)
data[features_ohe.columns] = features_ohe
data

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,...,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_no_info,NotRepaired_yes
0,2016-03-24 11:52:17,480,petrol,1993,manual,0,golf,150000,0,petrol,...,0,0,0,0,0,0,1,0,1,0
1,2016-03-24 10:58:45,18300,gasoline,2011,manual,190,no_info,125000,5,gasoline,...,0,0,0,0,0,0,0,0,0,1
2,2016-03-14 12:52:21,9800,gasoline,2004,auto,163,grand,125000,8,gasoline,...,0,0,0,0,0,0,0,0,1,0
3,2016-03-17 16:54:04,1500,petrol,2001,manual,75,golf,150000,6,petrol,...,0,0,0,0,0,0,1,0,0,0
4,2016-03-31 17:25:20,3600,gasoline,2008,manual,69,fabia,90000,7,gasoline,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354363,2016-03-27 20:36:20,1150,petrol,2000,manual,0,zafira,150000,3,petrol,...,0,0,0,0,0,0,0,0,0,0
354365,2016-03-14 17:48:27,2200,no_info,2005,no_info,0,no_info,20000,1,no_info,...,0,1,0,0,0,0,0,0,1,0
354366,2016-03-05 19:56:21,1199,petrol,2000,auto,101,fortwo,125000,3,petrol,...,1,0,0,0,0,0,0,0,0,0
354367,2016-03-19 18:57:12,9200,gasoline,1996,manual,102,transporter,150000,3,gasoline,...,0,0,0,0,0,0,1,0,0,0


In [12]:
features_columns = ['RegistrationYear','Power','Kilometer','RegistrationMonth','NumberOfPictures','time_in_use'] + list(features_ohe.columns)
features_columns

['RegistrationYear',
 'Power',
 'Kilometer',
 'RegistrationMonth',
 'NumberOfPictures',
 'time_in_use',
 'VehicleType_electric',
 'VehicleType_gasoline',
 'VehicleType_hybrid',
 'VehicleType_lpg',
 'VehicleType_no_info',
 'VehicleType_other',
 'VehicleType_petrol',
 'Gearbox_manual',
 'Gearbox_no_info',
 'Model_145',
 'Model_147',
 'Model_156',
 'Model_159',
 'Model_1_reihe',
 'Model_1er',
 'Model_200',
 'Model_2_reihe',
 'Model_300c',
 'Model_3_reihe',
 'Model_3er',
 'Model_4_reihe',
 'Model_500',
 'Model_5_reihe',
 'Model_5er',
 'Model_601',
 'Model_6_reihe',
 'Model_6er',
 'Model_7er',
 'Model_80',
 'Model_850',
 'Model_90',
 'Model_900',
 'Model_9000',
 'Model_911',
 'Model_a1',
 'Model_a2',
 'Model_a3',
 'Model_a4',
 'Model_a5',
 'Model_a6',
 'Model_a8',
 'Model_a_klasse',
 'Model_accord',
 'Model_agila',
 'Model_alhambra',
 'Model_almera',
 'Model_altea',
 'Model_amarok',
 'Model_antara',
 'Model_arosa',
 'Model_astra',
 'Model_auris',
 'Model_avensis',
 'Model_aveo',
 'Model_ayg

Графы: DateCrawled, PostalCode и LastSeen, я не беру в критерии от которых зивисит цена машины, так как информация не касающаяся машины не влияет на её цену.
Графу DateCreated, я заменил на графу показывающею сколько лет машина была в экспулатации до выставления на продажу.

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 323224 entries, 0 to 354368
Columns: 324 entries, DateCrawled to NotRepaired_yes
dtypes: int64(8), object(9), uint8(307)
memory usage: 139.0+ MB


In [14]:
target = data['Price']
features = data[features_columns]
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)
features_train_no_ohe, features_valid_no_ohe, target_train_no_ohe, target_valid_no_ohe = train_test_split(
    features_no_ohe, target_no_ohe, test_size=0.25, random_state=12345)

# 2. Обучение моделей

In [15]:
def score(model, target = target_train, features = features_train):
    start_time = timeit.default_timer()
    score = mean_squared_error(target, model.predict(features))**(1/2)
    predict_time = timeit.default_timer() - start_time
    return score, predict_time
def fit(model):
    start_time = timeit.default_timer()
    model.fit(features_train,target_train)
    fit_time = timeit.default_timer() - start_time
    return fit_time

## CatBoostRegressor_no_ohe

In [19]:
parametrs = { 'learning_rate': [0.1*i for i in range(5,11)],
              'iterations': range(60,110,10),}
cats = CatBoostRegressor()
grid = GridSearchCV(cats, parametrs)
grid.fit(features_valid_no_ohe,target_valid_no_ohe, silent= True, cat_features = ['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired'])
grid.best_params_

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


{'iterations': 100, 'learning_rate': 0.6000000000000001}

In [20]:
cat_no_ohe = CatBoostRegressor(iterations = 100, learning_rate = 0.6000000000000001, depth = 10)
start_time = timeit.default_timer()
cat_no_ohe.fit(features_train_no_ohe,target_train_no_ohe, silent= True, cat_features = ['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired'])
cat_no_ohe_fit_time = timeit.default_timer() - start_time
print('время обучения:', cat_no_ohe_fit_time)
Score, time  = score(cat_no_ohe,target_train_no_ohe, features_train_no_ohe)
print('RMSE score:', Score)
print('время предсказания:', time)

время обучения: 59.59664465300739
RMSE score: 1509.3750414435676
время предсказания: 0.9507800182327628


## CatBoostRegressor 

In [21]:
#%%time
cat = CatBoostRegressor(iterations = 500)
start_time = timeit.default_timer()
cat.fit(features_train,target_train, silent= True)
cat_fit_time = timeit.default_timer() - start_time
print('время обучения:', cat_fit_time)
Score, time  = score(cat)
print('RMSE score:', Score)
print('время предсказания:', time)

время обучения: 92.6651452369988
RMSE score: 1842.074618057925
время предсказания: 0.40655714366585016


## LGBMRegressor

In [22]:
LGBM = LGBMRegressor(iterations = 5, num_leaves = 60, n_estimators=150 )
LGBM_fit_time = fit(LGBM)
print('время обучения:', LGBM_fit_time)
Score, time  = score(LGBM)
print('RMSE score:', Score)
print('время предсказания:', time)

время обучения: 20.079794286750257
RMSE score: 1610.2658693276849
время предсказания: 7.4046037988737226


## DummyRegressor

In [23]:
dummy_regr = DummyRegressor(strategy="median")
dummy_regr_fit_time = fit(dummy_regr)
print('время обучения:', dummy_regr_fit_time)
Score, time  = score(dummy_regr)
print('RMSE score:', Score)
print('время предсказания:', time)

время обучения: 0.00428478978574276
RMSE score: 4860.020907973153
время предсказания: 0.002259279601275921


## LinearRegression

In [24]:
LinearRegr = LinearRegression()
LinearRegr_fit_time = fit(LinearRegr)
print('время обучения:', LinearRegr_fit_time)
Score, time  = score(LinearRegr)
print('RMSE score:', Score)
print('время предсказания:', time)

время обучения: 26.36878447048366
RMSE score: 2897.383156746465
время предсказания: 0.7032305020838976


# 3. Анализ моделей

## LinearRegression

In [25]:
LinearRegr_score, LinearRegr_time  = score(LinearRegr,target = target_valid,features = features_valid )
print('RMSE score:', LinearRegr_score)
print('время предсказания:', LinearRegr_time)

RMSE score: 2883.5556196208004
время предсказания: 0.1990299802273512


## LGBMRegressor

In [26]:
LGBM_score, LGBM_time  = score(LGBM,target = target_valid,features = features_valid )
print('RMSE score:', LGBM_score)
print('время предсказания:', LGBM_time)

RMSE score: 1672.8490518554756
время предсказания: 2.5060295034199953


лучше всего показала себя как на тестовой выборке, так и на обучающей

## CatBoostRegressor

In [27]:
cat_score, cat_time  = score(cat,target = target_valid,features = features_valid )
print('RMSE score:', cat_score)
print('время предсказания:', cat_time)

RMSE score: 1865.1447632112015
время предсказания: 0.1879817796871066


показала себя хуже предыдущей модели, но лучше дамми модели


## CatBoostRegressor_no_ohe

In [28]:
cat_no_ohe_score, cat_no_ohe_time  = score(cat_no_ohe,target = target_valid_no_ohe,features = features_valid_no_ohe )
print('RMSE score:', cat_no_ohe_score)
print('время предсказания:', cat_no_ohe_time)

RMSE score: 1681.4196477558974
время предсказания: 0.38633723463863134


# Сравнение моделей

In [29]:
columns = ['RMSE_score','predict_time','fit_time', 'total_time']
index = ['LinearRegression','LGBMRegressor','CatBoostRegressor','CatBoostRegressor_no_ohe']
final_data = [[LinearRegr_score,LinearRegr_time,LinearRegr_fit_time,LinearRegr_time + LinearRegr_fit_time],
              [LGBM_score,LGBM_time,LGBM_fit_time,LGBM_time + LGBM_fit_time],
             [cat_score,cat_time,cat_fit_time,cat_time + cat_fit_time],
              [cat_no_ohe_score,cat_no_ohe_time,cat_no_ohe_fit_time,cat_no_ohe_time + cat_no_ohe_fit_time]]
pd.DataFrame(columns = columns, index = index, data = final_data)

,RMSE_score,predict_time,fit_time,total_time
LinearRegression,2883.555620,0.199030,26.368784,26.567814
LGBMRegressor,1672.849052,2.506030,20.079794,22.585824
CatBoostRegressor,1865.144763,0.187982,92.665145,92.853127
CatBoostRegressor_no_ohe,1681.419648,0.386337,59.596645,59.982982


модель LGBMRegressor показывает лучших результат по качеству, но проваливаетcя по времени предсказания, обучается быстро. Модель LinearRegression предсказывает и учится быстрее, но предсказываеь хуже всего. CatBoostRegressor_no_ohe учится не самым быстрым образом, показывает хорошую скорость скорость предсказания и хороший результат, является фаворитом.